In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

from instance_selection.operator.metrics import calculate_gmean_mauc
from mlp_sklearn import MLPClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder

from instance_selection.parameter.parameter import *
import scipy.io as sio  # 从.mat文件中读取数据集

# 测试代码
if __name__ == "__main__":
    DATASET = Nursery  # 数据集名称（包含对应参数的字典形式）
    datasetname = DATASET['DATASETNAME'].split('.')[0]

    # 加载、划分数据集
    mat_data = sio.loadmat('../../data/dataset/' + DATASET['DATASETNAME'])
    x = mat_data['X']
    y = mat_data['Y'][:, 0]  # mat_data['Y']得到的形状为[n,1]，通过[:,0]，得到形状[n,]
    # 对y使用one-hot编码
    encoder = OneHotEncoder(sparse_output=False)
    y_onehot = encoder.fit_transform(y)
    x_train, x_test, y_train, y_test = train_test_split(x, y_onehot, test_size=0.3, random_state=RANDOM_SEED)  # 划分数据集
    scaler = StandardScaler()  # 数据的标准化
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)

    # 定义网络参数
    input_size = x.shape[1]
    hidden_size = DATASET['HIDDEN_SIZE']
    output_size = np.unique(y_train).size
    learning_rate = DATASET['LEARNING_RATE']

    # 创建MLP模型
    model = MLPClassifier(hidden_layer_sizes=(DATASET['HIDDEN_SIZE'],), max_iter=DATASET['MAX_ITER'],
                          random_state=RANDOM_SEED, learning_rate_init=DATASET['LEARNING_RATE'])

    model.fit(x_train, y_train)
    y_pred_prob = model.predict_proba(x_test)
    gmean, mauc, recall_per_class = calculate_gmean_mauc(y_pred_prob, y_test)
    print(f"G-Mean: {gmean:.4f}, M-AUC: {mauc:.4f}, Recall per class: {recall_per_class}")


In [ ]:
import numpy as np


class MLP:
    def __init__(self, input_size, hidden_sizes, output_size, learning_rate=0.01):
        # 初始化网络参数（权重和偏置）
        self.learning_rate = learning_rate
        self.layers = []
        self.biases = []

        # 初始化每一层的权重和偏置
        layer_sizes = [input_size] + hidden_sizes + [output_size]
        for i in range(len(layer_sizes) - 1):
            self.layers.append(np.random.randn(layer_sizes[i], layer_sizes[i + 1]) * 0.1)
            self.biases.append(np.zeros((1, layer_sizes[i + 1])))

    def relu(self, x):
        return np.maximum(0, x)

    def relu_derivative(self, x):
        return (x > 0).astype(float)

    def softmax(self, x):
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)

    def forward(self, x):
        # 存储每一层的中间结果
        self.zs = []  # 线性变换结果
        self.activations = [x]  # 激活后的结果

        for i in range(len(self.layers)):
            z = np.dot(self.activations[-1], self.layers[i]) + self.biases[i]
            self.zs.append(z)
            if i == len(self.layers) - 1:  # 输出层使用softmax
                activation = self.softmax(z)
            else:  # 隐藏层使用ReLU
                activation = self.relu(z)
            self.activations.append(activation)

        return self.activations[-1]

    def backward(self, x, y):
        # 前向传播获取预测值
        output = self.forward(x)

        # 计算输出层的误差
        m = y.shape[0]  # 样本数
        dz = output - y  # 输出层误差（交叉熵损失的梯度）

        for i in reversed(range(len(self.layers))):
            dw = np.dot(self.activations[i].T, dz) / m
            db = np.sum(dz, axis=0, keepdims=True) / m
            if i > 0:
                da = np.dot(dz, self.layers[i].T)
                dz = da * self.relu_derivative(self.zs[i - 1])

            # 更新权重和偏置
            self.layers[i] -= self.learning_rate * dw
            self.biases[i] -= self.learning_rate * db

    def train_one_sample(self, x, y):
        # 将输入 x 和 y 转换为行向量形式
        x = x.reshape(1, -1)
        y = y.reshape(1, -1)
        self.backward(x, y)

    def predict(self, x):
        # 前向传播获取输出
        output = self.forward(x)
        return np.argmax(output, axis=1)

    def predict_prob(self, x):
        """
        返回每个样本的预测概率分布。
        参数：
        - x: 输入数据，形状为 (n_samples, n_features)
        返回值：
        - probs: 预测概率分布，形状为 (n_samples, n_classes)
        """
        return self.forward(x)


# 测试MLP
if __name__ == "__main__":
    # 创建一个MLP，输入层大小为2，隐藏层大小为[4, 4]，输出层大小为2
    mlp = MLP(input_size=2, hidden_sizes=[4, 4], output_size=2, learning_rate=0.01)

    # 创建简单的数据集（逻辑回归任务）
    X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
    Y = np.array([[1, 0], [0, 1], [0, 1], [1, 0]])  # 独热编码标签

    # 训练模型，每次输入一个样本
    epochs = 1000
    for epoch in range(epochs):
        for i in range(X.shape[0]):
            mlp.train_one_sample(X[i], Y[i])

    # 测试模型
    predictions = mlp.predict(X)
    print("Predictions:", predictions)


### 模型的训练与测试

In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler


class MLP:
    def __init__(self, input_size, hidden_sizes, output_size, learning_rate=0.01):
        # 初始化网络参数（权重和偏置）
        self.learning_rate = learning_rate
        self.layers = []
        self.biases = []

        # 初始化每一层的权重和偏置
        layer_sizes = [input_size] + hidden_sizes + [output_size]
        for i in range(len(layer_sizes) - 1):
            self.layers.append(np.random.randn(layer_sizes[i], layer_sizes[i + 1]) * 0.1)
            self.biases.append(np.zeros((1, layer_sizes[i + 1])))

    def relu(self, x):
        return np.maximum(0, x)

    def relu_derivative(self, x):
        return (x > 0).astype(float)

    def softmax(self, x):
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)

    def forward(self, x):
        # 存储每一层的中间结果
        self.zs = []  # 线性变换结果
        self.activations = [x]  # 激活后的结果

        for i in range(len(self.layers)):
            z = np.dot(self.activations[-1], self.layers[i]) + self.biases[i]
            self.zs.append(z)
            if i == len(self.layers) - 1:  # 输出层使用softmax
                activation = self.softmax(z)
            else:  # 隐藏层使用ReLU
                activation = self.relu(z)
            self.activations.append(activation)

        return self.activations[-1]

    def backward(self, x, y):
        # 前向传播获取预测值
        output = self.forward(x)

        # 计算输出层的误差
        m = y.shape[0]  # 样本数
        dz = output - y  # 输出层误差（交叉熵损失的梯度）

        for i in reversed(range(len(self.layers))):
            dw = np.dot(self.activations[i].T, dz) / m
            db = np.sum(dz, axis=0, keepdims=True) / m
            if i > 0:
                da = np.dot(dz, self.layers[i].T)
                dz = da * self.relu_derivative(self.zs[i - 1])

            # 更新权重和偏置
            self.layers[i] -= self.learning_rate * dw
            self.biases[i] -= self.learning_rate * db

    def train_one_sample(self, x, y):
        # 将输入 x 和 y 转换为行向量形式
        x = x.reshape(1, -1)
        y = y.reshape(1, -1)
        self.backward(x, y)

    def predict_prob(self, x):
        return self.forward(x)

    def predict(self, x):
        probs = self.predict_prob(x)
        return np.argmax(probs, axis=1)


# 加载鸢尾花数据集
iris = load_iris()
X = iris.data  # 特征
y = iris.target  # 标签

# 数据预处理
# 将标签转换为独热编码
encoder = OneHotEncoder(sparse_output=False)
y_onehot = encoder.fit_transform(y.reshape(-1, 1))

# 标准化特征
scaler = StandardScaler()
X = scaler.fit_transform(X)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.3, random_state=42)

# 创建 MLP 模型
input_size = X_train.shape[1]  # 特征维度
hidden_sizes = [10, 8]  # 两个隐藏层，大小分别为 10 和 8
output_size = y_train.shape[1]  # 类别数
mlp = MLP(input_size=input_size, hidden_sizes=hidden_sizes, output_size=output_size, learning_rate=0.01)

# 训练模型
epochs = 1000
for epoch in range(epochs):
    for i in range(X_train.shape[0]):
        mlp.train_one_sample(X_train[i], y_train[i])
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch + 1}/{epochs} completed")

# 测试模型
y_test_probs = mlp.predict_prob(X_test)
y_test_preds = mlp.predict(X_test)

# 计算测试集准确率
y_test_labels = np.argmax(y_test, axis=1)
accuracy = np.mean(y_test_preds == y_test_labels)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


In [1]:
# 生成1000条特征数据x和对应的标签y(0-5)
import numpy as np

x = np.random.rand(20, 2)
y = np.random.randint(0, 6, size=100)
unique_elements, _ = np.unique(y, return_counts=True)
# 构造每个类别的索引列表
class_indices = {element: np.where(y == element)[0] for element in unique_elements}
# 输出类别以及索引列表
for element, indices in class_indices.items():
    print(f"Class {element}: {indices}")

Class 0: [ 2  4 11 14 21 24 27 32 38 40 43 46 62 72 75 82 92]
Class 1: [ 9 25 31 33 37 41 53 71 74 76 77 84 86 95]
Class 2: [12 30 36 39 45 47 50 51 54 56 57 58 60 66 67 69 93]
Class 3: [ 1  6  7 10 16 17 26 44 48 59 68 79 81 83 88 89 90 91 98]
Class 4: [ 3  5 13 15 23 34 35 42 49 61 73 78 80 87 96 99]
Class 5: [ 0  8 18 19 20 22 28 29 52 55 63 64 65 70 85 94 97]


In [ ]:
from instance_selection.operator.constraint import individuals_constraints_in_classes
# 生成2个长度为1000的list，list元素是0


import numpy as np
inds = np.zeros((2,100))
inds2 = individuals_constraints_in_classes(inds, x, y)

### MLP-SKLEARN

In [12]:
from utils.dataset_utils import get_distribution, k_fold_cross_validation
from instance_selection.parameter.parameter import *  # 导入参数的设定
from instance_selection.operator.metrics import calculate_gmean_mauc, calculate_average_accuracy, \
    calculate_average_gmean_mauc, calculate_accuracy
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.base import clone
import scipy.io as sio  # 从.mat文件中读取数据集
import random
from deap import tools
import warnings
import numpy as np

warnings.filterwarnings("ignore")  # 忽略警告

DATASET = Satellite

# 数据集名称（包含对应参数的字典形式）
datasetname = DATASET['DATASETNAME'].split('.')[0]
mat_data = sio.loadmat(IMBALANCED_DATASET_PATH + DATASET['DATASETNAME'])  # 加载、划分数据集
x = mat_data['X']
y = mat_data['Y'][:, 0]  # mat_data['Y']得到的形状为[n,1]，通过[:,0]，得到形状[n,]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=RANDOM_SEED)  # 划分数据集
scaler = StandardScaler()  # 数据的标准化
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)



model = MLPClassifier(hidden_layer_sizes=(DATASET['HIDDEN_SIZE'],), max_iter=DATASET['MAX_ITER'],
                      random_state=RANDOM_SEED, learning_rate_init=DATASET['LEARNING_RATE'],solver='adam')


model.fit(x_train, y_train)

# 将概率转化为预测结果
y_pred = model.predict(x_test)
y_pred_proba = model.predict_proba(x_test)

print(calculate_gmean_mauc(y_pred_proba, y_test))

(0.845994, 0.978635, array([0.96283784, 0.97080292, 0.82142857, 0.62307692, 0.89208633,
       0.85901639]))
